In [1]:
import numpy as np
import struct
import SimpleITK as sitk

filename = "bdf.bdf"

with open(filename, "rb") as f:
    data = f.read()

size = len(data)

image_size = []
spacing = []

for i in range(0,12,4):
    tmp = data[i:i+4]
    number = struct.unpack('<I', tmp)
    image_size.append(number[0])

for i in range(12,24,4):
    tmp = data[i:i+4]
    number = struct.unpack('f', tmp)
    spacing.append(number[0])

print("array size ",size)
print("image_size", image_size)
print("spacing", spacing)

assert np.prod(image_size)*4*3 + 24 == size

vector = []
vec_arr = []
count = 0

for i in range(24,size,4):
    tmp = data[i:i+4]


    number = struct.unpack('f', tmp)
    
    vector.append(number)
    count+=1
    
    if count ==3:
        vec_arr.append(vector)
        count = 0
        vector = []
        


array size  402653208
image_size [512, 512, 128]
spacing [0.9699999690055847, 0.9699999690055847, 2.5]


In [2]:
assert len(vec_arr) == np.prod(image_size)

vec_arr = np.array(vec_arr)
vec_arr = np.reshape(vec_arr, (*image_size[::-1], 3))

In [3]:


def image_from_array(array, origin, spacing):
    image = sitk.GetImageFromArray(array, isVector=True)
    image.SetSpacing(spacing)
    image.SetOrigin(origin)
    return image

In [4]:
image = image_from_array(vec_arr, (0,0,0),spacing  )

In [5]:
sitk.WriteImage(image, "image_test.nrrd")

transform = sitk.DisplacementFieldTransform(image)
sitk.WriteTransform(transform, 'transform_test.tfm')